In [1]:
!pip install gensim
!pip install nltk

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [4]:
import pandas as pd
import numpy as np
import torch

data = pd.read_csv('https://www.ksi.mff.cuni.cz/~peska/vyuka/nprg065/booksummaries.txt', sep="\t", header=None)

print(data.head())

mydata = {}


for i in range(len(data)):
   mydata[data.iloc[i,2]] = data.iloc[i,6]


i=0
for item in mydata.items():
  if(i > 10):
    pass
  else:
    print(item)
  i=i+1


      0        1                                          2                3  \
0   620  /m/0hhy                                Animal Farm    George Orwell   
1   843  /m/0k36                         A Clockwork Orange  Anthony Burgess   
2   986  /m/0ldx                                 The Plague     Albert Camus   
3  1756  /m/0sww  An Enquiry Concerning Human Understanding       David Hume   
4  2080  /m/0wkt                       A Fire Upon the Deep     Vernor Vinge   

            4                                                  5  \
0  1945-08-17  {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...   
1        1962  {"/m/06n90": "Science Fiction", "/m/0l67h": "N...   
2        1947  {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...   
3         NaN                                                NaN   
4         NaN  {"/m/03lrw": "Hard science fiction", "/m/06n90...   

                                                   6  
0   Old Major, the old boar on the Manor Farm, ca...  


In [ ]:
print(word_tokenize(mydata['Animal Farm']))
print(TaggedDocument((word_tokenize(mydata['Animal Farm'])), tags=['Animal Farm']))

['Old', 'Major', ',', 'the', 'old', 'boar', 'on', 'the', 'Manor', 'Farm', ',', 'calls', 'the', 'animals', 'on', 'the', 'farm', 'for', 'a', 'meeting', ',', 'where', 'he', 'compares', 'the', 'humans', 'to', 'parasites', 'and', 'teaches', 'the', 'animals', 'a', 'revolutionary', 'song', ',', "'Beasts", 'of', 'England', "'", '.', 'When', 'Major', 'dies', ',', 'two', 'young', 'pigs', ',', 'Snowball', 'and', 'Napoleon', ',', 'assume', 'command', 'and', 'turn', 'his', 'dream', 'into', 'a', 'philosophy', '.', 'The', 'animals', 'revolt', 'and', 'drive', 'the', 'drunken', 'and', 'irresponsible', 'Mr', 'Jones', 'from', 'the', 'farm', ',', 'renaming', 'it', '``', 'Animal', 'Farm', "''", '.', 'They', 'adopt', 'Seven', 'Commandments', 'of', 'Animal-ism', ',', 'the', 'most', 'important', 'of', 'which', 'is', ',', '``', 'All', 'animals', 'are', 'equal', "''", '.', 'Snowball', 'attempts', 'to', 'teach', 'the', 'animals', 'reading', 'and', 'writing', ';', 'food', 'is', 'plentiful', ',', 'and', 'the', 'fa

In [ ]:
#prepare the data for doc2vec, build and save a doc2vec model
train_doc2vec = [TaggedDocument((word_tokenize(mydata[t])), tags=[t]) for t in mydata.keys()]

print(train_doc2vec[0])

model = Doc2Vec(vector_size=50, alpha=0.025, min_count=10, dm =1, epochs=40)

model.build_vocab(train_doc2vec)

model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")

TaggedDocument<['Old', 'Major', ',', 'the', 'old', 'boar', 'on', 'the', 'Manor', 'Farm', ',', 'calls', 'the', 'animals', 'on', 'the', 'farm', 'for', 'a', 'meeting', ',', 'where', 'he', 'compares', 'the', 'humans', 'to', 'parasites', 'and', 'teaches', 'the', 'animals', 'a', 'revolutionary', 'song', ',', "'Beasts", 'of', 'England', "'", '.', 'When', 'Major', 'dies', ',', 'two', 'young', 'pigs', ',', 'Snowball', 'and', 'Napoleon', ',', 'assume', 'command', 'and', 'turn', 'his', 'dream', 'into', 'a', 'philosophy', '.', 'The', 'animals', 'revolt', 'and', 'drive', 'the', 'drunken', 'and', 'irresponsible', 'Mr', 'Jones', 'from', 'the', 'farm', ',', 'renaming', 'it', '``', 'Animal', 'Farm', "''", '.', 'They', 'adopt', 'Seven', 'Commandments', 'of', 'Animal-ism', ',', 'the', 'most', 'important', 'of', 'which', 'is', ',', '``', 'All', 'animals', 'are', 'equal', "''", '.', 'Snowball', 'attempts', 'to', 'teach', 'the', 'animals', 'reading', 'and', 'writing', ';', 'food', 'is', 'plentiful', ',', 'a

In [ ]:
#Use the model to look for similar texts
model= Doc2Vec.load("d2v.model")

#This is a sentence from the summary of “Animal Farm” on Wikipedia: https://en.wikipedia.org/wiki/Animal_Farm
story_sample1 = """
Napoleon enacts changes to the governance structure of the farm, replacing meetings with a committee of pigs who will run the farm.
 """
# part of story sentences from  Wikipedia: https://en.wikipedia.org/wiki/Children_of_Dune
story_sample2 = """
Meanwhile, a new religious figure called "The Preacher" has risen in the desert, rallying against the religious government's injustices and the changes among the Fremen. Some Fremen believe he is Paul Atreides.
 """
new_vector1 = model.infer_vector(word_tokenize(story_sample1))
new_vector2 = model.infer_vector(word_tokenize(story_sample2))

sims1 = model.docvecs.most_similar([new_vector1])
print(sims1)

sims2 = model.docvecs.most_similar([new_vector2])
print(sims2)

[('Animal Farm', 0.7186821699142456), ('The Wild Irish Girl', 0.7109571695327759), ('The Big Country', 0.6796502470970154), ('Texas Fever', 0.6794539093971252), ("What's Mine's Mine", 0.6722161173820496), ('Tros of Samothrace', 0.6581621766090393), ('Magic Mirror', 0.6522628664970398), ('The Woods Are Dark', 0.6516110897064209), ('Poor White', 0.6423494815826416), ('The Singapore Grip', 0.6381418108940125)]
[('Children of Dune', 0.7691787481307983), ('Paul of Dune', 0.7628101110458374), ('Dune', 0.7237648963928223), ('Dune Messiah', 0.7173170447349548), ("Fool's Bargain", 0.7156513333320618), ('The Winds of Dune', 0.7026625871658325), ('Bearers of the Black Staff', 0.6872473955154419), ('Autumn Mist', 0.6800292134284973), ('Dune: House Corrino', 0.6713364124298096), ('Tieta do Agreste', 0.6676875352859497)]


<ipython-input-19-ba802be61b85>:14: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims1 = model.docvecs.most_similar([new_vector1])
<ipython-input-19-ba802be61b85>:17: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims2 = model.docvecs.most_similar([new_vector2])
